# Imports

In [1]:
import pandas as pd
import numpy as np
import math

import io
import requests

import time

import gspread
from oauth2client.service_account import ServiceAccountCredentials

scope = "https://spreadsheets.google.com/feeds"
credentials = ServiceAccountCredentials.from_json_keyfile_name("matcher-272116-8801bce55bcb.json", scope)
gs = gspread.authorize(credentials)

data = None

# Load Journey Data

In [2]:
url = "https://www.anyvan.com/board/cyfe/express_interest_expresses"
s = requests.get(url).content

try :
    data = pd.read_csv(io.StringIO(s.decode("utf-8"))).sort_values(by = ["start_end"], ascending = False)
except :
    print("No journeys available")

No journeys available


In [3]:
url = "https://docs.google.com/spreadsheets/d/1o7jEklbPYk-7So0UALnt8dZCYlIbdyXhzsgm9MHf82I/edit#gid=0"

express_guaranteed_sheet = gs.open_by_url(url).worksheet("journeys").get_all_values()
data = pd.DataFrame(express_guaranteed_sheet[1:], columns = express_guaranteed_sheet[0])
data = data.replace("", np.nan)

# Returns

In [4]:
url = "https://docs.google.com/spreadsheets/d/1_syJVv1Uni8oSa9BhtAjVcl2EZYZmJn03VKg9vofcD4/edit#gid=0"

return_journey_converter_sheet = gs.open_by_url(url).worksheet("Input").get_all_values()[15:]
returns = pd.DataFrame(return_journey_converter_sheet[1:], columns = return_journey_converter_sheet[0])
returns = returns.drop(columns = returns.columns[16:])
returns["nickname"].replace("", float("NaN"), inplace = True)
returns = returns.dropna()

# Providers

In [5]:
sizes = {"Luton Van" : 20, 
         "LWB up to 4m" : 10.5,
         "MWB up to 3m" : 7.5,
         "Not applicable" : 12,
         "XLWB up 4m+" : 14.5,
         "SWB up to 2.4m" : 5.5,
         "Small Van" : 8}

In [6]:
url = "https://docs.google.com/spreadsheets/d/1t0L9sgsUt-BT7wJWBNZ4fNY-Bo5GNytXP1wmkejFVw0/edit#gid=1205069010"

TP_log_providers_sheet = gs.open_by_url(url).worksheet("format2").get_all_values()[7:]
TP_log_providers = pd.DataFrame(TP_log_providers_sheet[1:], columns = TP_log_providers_sheet[0])
providers = TP_log_providers.where(np.logical_and(TP_log_providers["vehicle"] != "N/A", np.logical_and(TP_log_providers["TP & REG"] != "AVStore #YN19FHD", np.logical_and(TP_log_providers["TP & REG"] != "1London #YN19FHF", TP_log_providers["TP & REG"] != "DWB91 #YN19FHD")))).dropna(how = "all").reset_index(drop = True).drop(columns = TP_log_providers.columns[-1])
date = providers.columns[-1]

providers["Size"] = [sizes[vehicle] for vehicle in providers["vehicle"]]
providers["National?"] = [int(math.ceil(55 + (100 - float(nuts2c)) / 2 + (2000 if code.endswith("N") else 0))) for nuts2c,code in zip(providers["nuts2c"], providers[date])]
providers["1Man"] = [1 if (code.startswith("1") or code.startswith("O")) else 0 for code in providers[date]]
providers["2Man"] = [2 if (code.startswith("2") or code.startswith("O")) else 0 for code in providers[date]]
providers["returns"] = [1 if TP in returns["name#reg"] else 0 for TP in providers["TP & REG"]]
providers["guaranteed"] = [1 if isreturn == 1 else -1 for isreturn in providers["returns"]]
providers["special code"] = ["2man-TL-LDN-cc" if (TwoMan == 2 and TL == "1") else "TL-LDN-cc" if (TL == "1") else "2man-LDN-cc" if (TwoMan == 2) else "LDN-cc" for TwoMan,TL in zip(providers["2Man"], providers["TL"])]
providers["capacity"] = [16 if size >= 16 else size for size in providers["Size"]]

providers = providers.sort_values(by = ["TP & REG"]).reset_index(drop = True)
providers["index"] = providers.index

# Expressing TPs

In [7]:
columns = ["TP", "lat", "lng", "expressions", "guaranteed", "boooked"]
expressing_TPs = pd.DataFrame(columns = columns)

if data is not None :
    for TP in data["name#reg"].dropna().unique() :
        data_TP = data.where(data["name#reg"] == TP).dropna(how = "all")
        lat = round(float(data_TP.iloc[0]["veh_lat"]), 2)
        lng = round(float(data_TP.iloc[0]["veh_lng"]), 2)
        expressions = len(data_TP.index)
        guaranteed = 1 if (TP in providers["TP & REG"] or TP in returns["name#reg"]) else 0
        booked = 1 if TP in returns["name#reg"] else 0
        expressing_TPs = expressing_TPs.append(pd.DataFrame([[TP, lat, lng, expressions, guaranteed, booked]], columns = columns))
else :
    print("No journeys available")
    
expressing_TPs = expressing_TPs.sort_values(by = ["TP"]).reset_index(drop = True)
expressing_TPs["index"] = expressing_TPs.index

# TPs

In [8]:
TP_namereg = np.append(providers["TP & REG"], expressing_TPs["TP"])
TP_lat = np.append(providers["Lat"], expressing_TPs["lat"])
TP_lng = np.append(providers["Lng"], expressing_TPs["lng"])
TP_TL = np.append(providers["TL"], np.full(len(expressing_TPs.index), np.nan))
TP_capacity = np.append(providers["Size"], np.full(len(expressing_TPs.index), np.nan))
TP_maxkm = np.append(providers["National?"], np.full(len(expressing_TPs.index), np.nan))
TP_1man = np.append(providers["1Man"], np.full(len(expressing_TPs.index), np.nan))
TP_2man = np.append(providers["2Man"], np.full(len(expressing_TPs.index), np.nan))
TP_guaranteed = np.append(providers["guaranteed"], expressing_TPs["guaranteed"])
TP_provider = np.append(np.ones(len(providers.index), dtype = bool), np.zeros(len(expressing_TPs.index), dtype = bool))

columns = ["TP", "lat", "lng", "TL", "capacity", "maxkm", "1man", "2man", "guaranteed", "provider"]
TPs = pd.DataFrame(zip(TP_namereg, TP_lat, TP_lng, TP_TL, TP_capacity, TP_maxkm, TP_1man, TP_2man, TP_guaranteed, TP_provider), columns = columns)
TPs = TPs.where(TPs["guaranteed"] <= 0).dropna(how = "all").sort_values(by = ["provider", "TP"], ascending = [False, True])

# Calculate iarray

In [9]:
def distance_funct(start_lat, start_lng, lat, lng) :
    return 2 * 6371 * np.arcsin(np.sqrt(np.sin((start_lat - lat) * np.pi / 360) ** 2 
                                           + np.cos(start_lat * np.pi / 180) 
                                           * np.cos(lat * np.pi / 180) 
                                           * np.sin((start_lng - lng) * np.pi / 360) ** 2))

In [24]:
start_time = time.time()

if data is not None :
    journeys = data["journey_id"].unique()
    iarray = np.zeros((len(TPs.index), len(journeys)), dtype = float)

    for i in range(len(journeys)) :
        journey = data.where(data["journey_id"] == journeys[i]).dropna(how = "all")
        
        for j in range(len(TPs.index)) :
            if int(TPs["guaranteed"].iloc[j]) == -1 :
                if TPs["TP"].iloc[j] in journey["name#reg"].values :
                    journey_subset = journey.where(journey["name#reg"] == TPs["TP"].iloc[j]).dropna(how = "all").iloc[0]
                    if float(journey_subset["min_dist"]) >= 5 :
                        iarray[j, i] = 5
                    else :
                        iarray[j, i] = float(journey_subset["min_dist"])
                else :
                    if float(journey["max_m3"].iloc[0]) <= float(TPs["capacity"].iloc[j]) :
                        if float(journey["start_end"].iloc[0]) <= float(TPs["maxkm"].iloc[j]) :
                            if int(journey["number_of_men"].iloc[0]) == int(TPs["1man"].iloc[j]) or int(journey["number_of_men"].iloc[0]) == int(TPs["2man"].iloc[j]) :
                                if int(journey["tail_lift"].iloc[0]) <= int(TPs["TL"].iloc[j]) :
                                    distance = distance_funct(float(journey["start_lat"].iloc[0]), float(journey["start_lng"].iloc[0]), float(TPs["lat"].iloc[j]), float(TPs["lng"].iloc[j]))
                                    if distance <= (float(TPs["maxkm"].iloc[j]) if float(TPs["maxkm"].iloc[j]) < 2000 else float(TPs["maxkm"].iloc[j]) - 2000) :
                                        iarray[j, i] = distance
            elif TPs["TP"].iloc[j] in journey["name#reg"].values :
                iarray[j, i] = float(journey.where(journey["name#reg"] == TPs["TP"].iloc[j]).dropna(how = "all")["min_dist"].iloc[0])
else :
    print("No journeys available")
                
print("Took {0:.3f} secs to run".format(time.time() - start_time))

Took 1.930 secs to run


In [37]:
list(zip(TPs["TP"],iarray.transpose()[np.argwhere(journeys == "33238")[0][0]]))

[('A1Van #FD61BLV', 0.0),
 ('Abeysaab #YD66NXM', 0.0),
 ('AntTrans #LR67KTU', 15.393330421768995),
 ('Apcybul #GJ15JVM', 26.052854440188657),
 ('BARTHA85 #KW14VFL', 0.0),
 ('BrockRem #CP09UFS', 0.0),
 ('BuzzBogn #TS54BUZ', 0.0),
 ('CODE2019 #YG61LRK', 0.0),
 ('Cieslik8 #CE17ZZF', 0.0),
 ('Core2012 #GN06PPZ', 28.56693185021415),
 ('Crisrey1 #KV59VGM', 9.59429253539582),
 ('DJB #FD17JBV', 0.0),
 ('DWB911 #YM66HRP', 2.72),
 ('DannyB17 #LJ15VHL', 0.0),
 ('Dapitan7 #688KM', 0.0),
 ('Davies56 #RV08OOY', 0.0),
 ('DawPol #YE16UCL', 0.0),
 ('Dprroni #KM69HLA', 0.0),
 ('ERmove #MO09VER', 0.0),
 ('GMS2019 #LC57KVA', 29.163850686319517),
 ('Gaben456 #MK16OHU', 0.0),
 ('Gelmoves #CA69YLB', 23.521770042517787),
 ('Gsavas #KN60YOR', 0.0),
 ('Indulis #MW56NGJ', 0.0),
 ('ItemDrop #DO03DOE', 0.0),
 ('JSJMOVE #KU62FFM', 0.0),
 ('Kamelia7 #BW65PXC', 0.0),
 ('Kevvan #DU16FFJ', 18.298029952402274),
 ('LCS2019 #BK11VBZ11', 0.0),
 ('Ltgibb5 #CN67DYF', 0.0),
 ('Mal2020 #DE59KZY', 0.0),
 ('Maps85 #HG59FDU', 0.0

# Done